In [3]:
from talk2car import Talk2Car
import csv
import json
import pickle
import sys
sys.path.append("")

csv_file = "/media/thierry/First/datasets/Cityscapes_Gaze/descriptions_gaze.csv"

allPunct = ["?", "!", "\\", "/", ")", "(", ".", ",", ";", ":"]
def tokenize(text, ignoredPuncts=["?", "!", "\\", "/", ")", "("],
             keptPuncts=[".", ",", ";", ":"], endPunct=[">", "<", ":"], delim=" ",
             clean=False, replacelistPre=dict(), replacelistPost=dict()):
    if clean:
        for word in replacelistPre:
            origText = text
            text = text.replace(word, replacelistPre[word])
            if (origText != text):
                print(origText)
                print(text)
                print("")
        for punct in endPunct:
            if text[-1] == punct:
                print(text)
                text = text[:-1]
                print(text)
                print("")
    for punct in keptPuncts:
        text = text.replace(punct, delim + punct + delim)
    for punct in ignoredPuncts:
        text = text.replace(punct, "")
    ret = text.lower().split(delim)
    if clean:
        origRet = ret
        ret = [replacelistPost.get(word, word) for word in ret]
        if origRet != ret:
            print(origRet)
            print(ret)
    ret = [t for t in ret if t != ""]
    return ret

class T2C(object):
    N_CLASSES = 23
    CLASSES = ('animal','human.pedestrian.adult', 'human.pedestrian.child',
                  'human.pedestrian.construction_worker', 'human.pedestrian.personal_mobility',
                  'human.pedestrian.police_officer', 'human.pedestrian.stroller', 'human.pedestrian.wheelchair',
                  'movable_object.barrier', 'movable_object.debris', 'movable_object.pushable_pullable',
                  'movable_object.trafficcone', 'static_object.bicycle_rack', 'vehicle.bicycle',
                  'vehicle.bus.bendy', 'vehicle.bus.rigid', 'vehicle.car', 'vehicle.construction','vehicle.emergency.ambulance',
                  'vehicle.emergency.police', 'vehicle.motorcycle', 'vehicle.trailer', 'vehicle.truck')

    MEAN = [123.68, 116.779, 103.939]   # R,G,B

    label_to_id = dict(map(reversed, enumerate(CLASSES)))
    id_to_label = dict(enumerate(CLASSES))
    
    

In [4]:

t2c_train = Talk2Car(version="train", dataroot="/export/home2/NoCsBack/hci/thierry/datasets/nuScenes/data/sets/nuscenes")
t2c_val = Talk2Car(version="val", dataroot="/export/home2/NoCsBack/hci/thierry/datasets/nuScenes/data/sets/nuscenes")
t2c_test = Talk2Car(version="test", dataroot="/export/home2/NoCsBack/hci/thierry/datasets/nuScenes/data/sets/nuscenes")

Loading NuScenes tables for version v1.0-trainval ...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 35.1 seconds.
Reverse indexing ...
Done reverse indexing in 9.7 seconds.
Loading NuScenes tables for version v1.0-trainval ...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 34.9 seconds.
Reverse indexing ...
Done reverse indexing in 10.2 seconds.
Loading NuScenes tables for version v1.0-trainval ...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 37.5 seconds.
Reverse indexing ...
Done reverse i

In [25]:
descr_train = json.load(open("/cw/liir/NoCsBack/testliir/datasets/refer_expr/Talk2Car/talk2car_descriptions_train.json", "r"))
descr_val = json.load(open("/cw/liir/NoCsBack/testliir/datasets/refer_expr/Talk2Car/talk2car_descriptions_val.json", "r"))
descr_test = json.load(open("/cw/liir/NoCsBack/testliir/datasets/refer_expr/Talk2Car/talk2car_descriptions_test.json", "r"))

In [26]:
list(descr_train.values())[0]

{'description': 'The black car parked on the right side of the street.',
 'attrs': ['right', 'black', 'parked', 'street'],
 'img': 'train_20.jpg',
 'bbox': [1003, 461, 239, 166],
 'class': 'vehicle.car'}

In [27]:
# Add description to t2c
def add_descr(ds, descrs):
    for c in ds.commands:
        if c.command_token not in descrs:
           c.description_data = None
        else:
            c.description_data = descrs[c.command_token] 
    

In [28]:
add_descr(t2c_train, descr_train)
add_descr(t2c_test, descr_test)
add_descr(t2c_val, descr_val)

In [29]:
t2c_train.commands[0]

Scene token: f92422ed4b4e427194a4958ccf15709a,
Frame token: c32d636e44604d77a1734386b3fe4a0d,
Box:         label: nan, score: nan, xyz: [-13.49, 0.43, 59.28], wlh: [0.81, 0.73, 1.96], rot axis: [-0.41, -0.64, 0.65], ang(degrees): -134.51, ang(rad): -2.35, vel: nan, nan, nan, name: human.pedestrian.adult, token: None

In [30]:
c = t2c_train.commands[0]
sd_rec = c.t2c.get('sample_data', c.frame_token)
_, boxes, camera_intrinsic = c.t2c.get_sample_data(sd_rec['token'])
[(t2c_train._transform_3d_to_2d_bbox(bbox.corners(), camera_intrinsic), bbox.token) for bbox in boxes], c.box

([((529, 458, 24, 41), '0183ed8a474f411f8a3394eb78df7838'),
  ((805, 461, 116, 95), '74f9ebe0551245a4b8c75baa56c94650'),
  ((1502, 453, 98, 70), '6ffbac85dc92440b863ab5ebccddb955'),
  ((1330, 464, 197, 62), '23d94b18989b4585acd689b687e3be7d'),
  ((1525, 451, 75, 76), '648702e2c9fd45e1a22cc1e4e93022b2'),
  ((761, 461, 83, 67), 'b625a075f5d647238446796e606b971e')],
 label: nan, score: nan, xyz: [-13.49, 0.43, 59.28], wlh: [0.81, 0.73, 1.96], rot axis: [-0.41, -0.64, 0.65], ang(degrees): -134.51, ang(rad): -2.35, vel: nan, nan, nan, name: human.pedestrian.adult, token: None)

In [31]:
T2C.label_to_id[t2c_train.commands[0].box.name]+1, t2c_train.commands[0].box.name

(2, 'human.pedestrian.adult')

In [32]:
import numpy as np

In [72]:
ds = []
out_dict = {
    "info": {"description": "Talk2Car",
             "url": "", "version": "1.0", "year": 2020,
             "contributor": "", "date_created": ""},
    "images": [],
    "licenses": [],
    "annotations": [],
    "categories": [{"supercategory": x.split(".")[0], "id": i+1, "name": x} for i,x in enumerate(T2C.CLASSES)],
}
imgs = {}
anns = []
pickled = []

abs_path = "/cw/liir/NoCsBack/testliir/datasets/refer_expr/Talk2Car/imgs/"

for command in t2c_train.commands + t2c_val.commands + t2c_test.commands:
    if command.description_data is None: continue
        
    img_path = abs_path+command.description_data["img"]
    
    name = command.description_data["img"]
    
    if name not in imgs:
        imgs[name] = {"license": 1, "file_name": name, "coco_url": img_path, "height": 900, "width": 1600,
                      "date_captured": "2013-11-21 23:06:41",
                      "flickr_url": img_path, "id": len(imgs)}

    sd_rec = command.t2c.get('sample_data', command.frame_token)
    _, boxes, camera_intrinsic = command.t2c.get_sample_data(sd_rec['token'])
    
    bbox = list(t2c_train._transform_3d_to_2d_bbox(command.box.corners(), camera_intrinsic))
    ann_id = None
    if bbox[2]  == 0 or bbox[3] == 0:
        print("WHAT")
        print(bbox)
        break
        
    box_2d = [(t2c_train._transform_3d_to_2d_bbox(bbox.corners(), camera_intrinsic), bbox) for bbox in boxes]
    # Add all objects in the image to the file
    for b in box_2d:
        
        if (b[0][2] == 0 or b[0][3] == 0 or b[0][2] < 10 or b[0][3] < 10) and not np.allclose(b[0], bbox):
            continue
        if np.allclose(b[0], bbox):
            ann_id = len(out_dict["annotations"])
            
        out_dict["annotations"].append({
                "image_id": imgs[name]["id"],
                "bbox": b[0],
                "category_id": T2C.label_to_id[b[1].name]+1,
                "id": len(out_dict["annotations"])
            })
      
    if ann_id == None:
        print([b[0] for b in box_2d])
        print(bbox)
        print(t2c_train._transform_3d_to_2d_bbox(command.box.corners(), camera_intrinsic))
        print("Breaking!!!")
        break
              
    split = command.description_data["img"].split("_")[0]
    descr = command.description_data["description"]
    pickled.append({'sent_ids': [len(pickled)], 'file_name': name,
                    'ann_id': ann_id, 'ref_id': ann_id,
                     'image_id': imgs[name]["id"], 'split': split, 
                    'sentences': [
                        {'tokens': tokenize(descr), 
                         'raw': command.description_data["description"],
                         'sent_id': len(pickled), 
                         'sent': command.description_data["description"]}
                    ],
                    'category_id': T2C.label_to_id[command.box.name]+1})


In [73]:
out_dict["annotations"][:10]

[{'image_id': 0, 'bbox': (529, 458, 24, 41), 'category_id': 2, 'id': 0},
 {'image_id': 0, 'bbox': (805, 461, 116, 95), 'category_id': 17, 'id': 1},
 {'image_id': 0, 'bbox': (1502, 453, 98, 70), 'category_id': 17, 'id': 2},
 {'image_id': 0, 'bbox': (1330, 464, 197, 62), 'category_id': 17, 'id': 3},
 {'image_id': 0, 'bbox': (1525, 451, 75, 76), 'category_id': 17, 'id': 4},
 {'image_id': 0, 'bbox': (761, 461, 83, 67), 'category_id': 17, 'id': 5},
 {'image_id': 1, 'bbox': (751, 485, 27, 31), 'category_id': 9, 'id': 6},
 {'image_id': 1, 'bbox': (686, 489, 59, 41), 'category_id': 9, 'id': 7},
 {'image_id': 1, 'bbox': (504, 505, 106, 63), 'category_id': 9, 'id': 8},
 {'image_id': 1, 'bbox': (47, 521, 79, 45), 'category_id': 9, 'id': 9}]

In [74]:
json.dumps(list(T2C.CLASSES))

'["animal", "human.pedestrian.adult", "human.pedestrian.child", "human.pedestrian.construction_worker", "human.pedestrian.personal_mobility", "human.pedestrian.police_officer", "human.pedestrian.stroller", "human.pedestrian.wheelchair", "movable_object.barrier", "movable_object.debris", "movable_object.pushable_pullable", "movable_object.trafficcone", "static_object.bicycle_rack", "vehicle.bicycle", "vehicle.bus.bendy", "vehicle.bus.rigid", "vehicle.car", "vehicle.construction", "vehicle.emergency.ambulance", "vehicle.emergency.police", "vehicle.motorcycle", "vehicle.trailer", "vehicle.truck"]'

In [75]:

out_dict['images'] = list(imgs.values())
json.dump(out_dict,
          open("/export/home2/NoCsBack/hci/thierry/netapp_thierry/attribute-2017/dataset/anns/original/ref2/talk2car/instances.json","w"))
pickle.dump(pickled, open("/export/home2/NoCsBack/hci/thierry/netapp_thierry/attribute-2017/dataset/anns/original/ref2/talk2car/refs(kul).p","wb"))


